# 05 · Model Evaluation
Load the saved model (from a run) and compute metrics/plots.

In [ ]:

import pandas as pd
import numpy as np
import mlflow
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support, confusion_matrix, RocCurveDisplay
import matplotlib.pyplot as plt

FEAT_CSV = "/mnt/data/IPO_Databricks_Notebooks/features.csv"
df = pd.read_csv(FEAT_CSV)

y = df["Delisted"].values
X = df.drop(columns=["Delisted","ipoDate","delistingDate"], errors="ignore")

# 👉 Replace with your MLflow run's model URI (or use last_run.txt if you store it)
# For demo, we'll just re-train quickly here to evaluate.
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

cat_cols = [c for c in X.columns if X[c].dtype == object]
num_cols = [c for c in X.columns if c not in cat_cols]

pre = ColumnTransformer([
    ("cats", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ("nums", StandardScaler(), num_cols)
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
model = Pipeline([("pre", pre), ("clf", LogisticRegression(max_iter=200))]).fit(X_train, y_train)

y_prob = model.predict_proba(X_test)[:,1]
y_pred = (y_prob >= 0.5).astype(int)

auc = roc_auc_score(y_test, y_prob)
pr, rc, f1, _ = precision_recall_fscore_support(y_test, y_pred, average="binary", zero_division=0)
cm = confusion_matrix(y_test, y_pred)

print("AUC:", auc, "Precision:", pr, "Recall:", rc, "F1:", f1)
print("Confusion matrix:\n", cm)

RocCurveDisplay.from_predictions(y_test, y_prob)
plt.show()
